In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/DaiVietGPT

/content/drive/.shortcut-targets-by-id/13a2vbhrAYcqpnJn_I8_GqY7JRilJNnQ7/DUTHackathon/DaiVietGPT


In [ ]:
!pip install -r requirements.txt
!pip install fastapi uvicorn python-multipart
!pip install nest-asyncio pyngrok
!pip install openai
!pip install firebase-admin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 953.8 kB/s eta 0:00:00


In [ ]:
import firebase_admin
from firebase_admin import credentials, storage

cred = credentials.Certificate("key.json")
firebase_admin.initialize_app(cred, {
    # gs://daivietgpt.appspot.com
    'storageBucket': 'daivietgpt.appspot.com'
})


In [ ]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

# from googletrans import Translator
from pprint import pprint
# from datasets import load_dataset
from huggingface_hub import notebook_login

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [ ]:
import torch, uuid
import os, sys, shutil
from src.utils.preprocess import CropAndExtract
from src.test_audio2coeff import Audio2Coeff
from src.facerender.animate import AnimateFromCoeff
from src.generate_batch import get_data
from src.generate_facerender_batch import get_facerender_data

from src.utils.init_path import init_path

from pydub import AudioSegment


def mp3_to_wav(mp3_filename,wav_filename,frame_rate):
    mp3_file = AudioSegment.from_file(file=mp3_filename)
    mp3_file.set_frame_rate(frame_rate).export(wav_filename,format="wav")


class SadTalker():

    def __init__(self, checkpoint_path='checkpoints', config_path='src/config', lazy_load=False, size = 256, preprocess='crop'):

        if torch.cuda.is_available() :
            device = "cuda"
        else:
            device = "cpu"

        self.device = device

        os.environ['TORCH_HOME']= checkpoint_path

        self.checkpoint_path = checkpoint_path
        self.config_path = config_path
        self.sadtalker_paths = init_path(self.checkpoint_path, self.config_path, size, False, preprocess)
        print(self.sadtalker_paths)

        self.audio_to_coeff = Audio2Coeff(self.sadtalker_paths, self.device)
        self.preprocess_model = CropAndExtract(self.sadtalker_paths, self.device)
        self.animate_from_coeff = AnimateFromCoeff(self.sadtalker_paths, self.device)

    def test(self, source_image, driven_audio, preprocess='crop',
        still_mode=False,  use_enhancer=False, batch_size=1, size=256,
        pose_style = 0, exp_scale=1.0,
        use_ref_video = False,
        ref_video = None,
        ref_info = None,
        use_idle_mode = False,
        length_of_audio = 0, use_blink=True,
        result_dir='result'):

        time_tag = str(uuid.uuid4())
        save_dir = os.path.join(result_dir, time_tag)
        # save_dir = result_dir
        os.makedirs(save_dir, exist_ok=True)

        input_dir = os.path.join(save_dir, 'input')
        os.makedirs(input_dir, exist_ok=True)

        print(source_image)
        pic_path = os.path.join(input_dir, os.path.basename(source_image))
        shutil.copy(source_image, input_dir)

        if driven_audio is not None and os.path.isfile(driven_audio):
            audio_path = os.path.join(input_dir, os.path.basename(driven_audio))

            #### mp3 to wav
            if '.mp3' in audio_path:
                mp3_to_wav(driven_audio, audio_path.replace('.mp3', '.wav'), 16000)
                audio_path = audio_path.replace('.mp3', '.wav')
            else:
                shutil.copy(driven_audio, input_dir)

        elif use_idle_mode:
            audio_path = os.path.join(input_dir, 'idlemode_'+str(length_of_audio)+'.wav') ## generate audio from this new audio_path
            from pydub import AudioSegment
            one_sec_segment = AudioSegment.silent(duration=1000*length_of_audio)  #duration in milliseconds
            one_sec_segment.export(audio_path, format="wav")
        else:
            print(use_ref_video, ref_info)
            assert use_ref_video == True and ref_info == 'all'

        if use_ref_video and ref_info == 'all': # full ref mode
            ref_video_videoname = os.path.basename(ref_video)
            audio_path = os.path.join(save_dir, ref_video_videoname+'.wav')
            print('new audiopath:',audio_path)
            # if ref_video contains audio, set the audio from ref_video.
            cmd = r"ffmpeg -y -hide_banner -loglevel error -i %s %s"%(ref_video, audio_path)
            os.system(cmd)

        os.makedirs(save_dir, exist_ok=True)

        #crop image and extract 3dmm from image
        first_frame_dir = os.path.join(save_dir, 'first_frame_dir')
        os.makedirs(first_frame_dir, exist_ok=True)
        first_coeff_path, crop_pic_path, crop_info = self.preprocess_model.generate(pic_path, first_frame_dir, preprocess, True, size)

        if first_coeff_path is None:
            raise AttributeError("No face is detected")

        if use_ref_video:
            print('using ref video for genreation')
            ref_video_videoname = os.path.splitext(os.path.split(ref_video)[-1])[0]
            ref_video_frame_dir = os.path.join(save_dir, ref_video_videoname)
            os.makedirs(ref_video_frame_dir, exist_ok=True)
            print('3DMM Extraction for the reference video providing pose')
            ref_video_coeff_path, _, _ =  self.preprocess_model.generate(ref_video, ref_video_frame_dir, preprocess, source_image_flag=False)
        else:
            ref_video_coeff_path = None

        if use_ref_video:
            if ref_info == 'pose':
                ref_pose_coeff_path = ref_video_coeff_path
                ref_eyeblink_coeff_path = None
            elif ref_info == 'blink':
                ref_pose_coeff_path = None
                ref_eyeblink_coeff_path = ref_video_coeff_path
            elif ref_info == 'pose+blink':
                ref_pose_coeff_path = ref_video_coeff_path
                ref_eyeblink_coeff_path = ref_video_coeff_path
            elif ref_info == 'all':
                ref_pose_coeff_path = None
                ref_eyeblink_coeff_path = None
            else:
                raise('error in refinfo')
        else:
            ref_pose_coeff_path = None
            ref_eyeblink_coeff_path = None

        #audio2ceoff
        if use_ref_video and ref_info == 'all':
            coeff_path = ref_video_coeff_path # self.audio_to_coeff.generate(batch, save_dir, pose_style, ref_pose_coeff_path)
        else:
            batch = get_data(first_coeff_path, audio_path, self.device, ref_eyeblink_coeff_path=ref_eyeblink_coeff_path, still=still_mode, idlemode=use_idle_mode, length_of_audio=length_of_audio, use_blink=use_blink) # longer audio?
            coeff_path = self.audio_to_coeff.generate(batch, save_dir, pose_style, ref_pose_coeff_path)

        #coeff2video
        data = get_facerender_data(coeff_path, crop_pic_path, first_coeff_path, audio_path, batch_size, still_mode=still_mode, preprocess=preprocess, size=size, expression_scale = exp_scale)
        return_path = self.animate_from_coeff.generate(data, save_dir,  pic_path, crop_info, enhancer='gfpgan' if use_enhancer else None, preprocess=preprocess, img_size=size)
        video_name = data['video_name']
        print(f'The generated video is named {video_name} in {save_dir}')

        # del self.preprocess_model
        # del self.audio_to_coeff
        # del self.animate_from_coeff

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

        import gc; gc.collect()
        return return_path




In [ ]:
MODEL_NAME = "vilm/vinallama-7b-chat"
# MODEL_NAME = "/content/results"
# tokenizer_dir = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

vinallama_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

vinallama_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
vinallama_tokenizer.pad_token = vinallama_tokenizer.eos_token

vinallama_model.gradient_checkpointing_enable()
vinallama_model = prepare_model_for_kbit_training(vinallama_model)

config = LoraConfig(
    r = 16,
    lora_alpha=32,
    target_modules = [
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout = 0.05,
    bias='none',
    task_type = "CAUSAL_LM"
)

vinallama_model = get_peft_model(vinallama_model, config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_config = vinallama_model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.5
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = vinallama_tokenizer.eos_token_id
# generation_config.pad_token_id = tokenizer.pad_token_id
generation_config.eos_token_id = vinallama_tokenizer.eos_token_id
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def generate_prompt(text) :
  # translator = Translator()
  # vn_prompt = translator.translate(data_point['prompt'], src ='en', dest = 'vi').text
  # vn_response = translator.translate(data_point['response'], src ='en', dest = 'vi').text
  # vn_prompt = data_point['prompt']
  # vn_response = data_point['response']
  return f"""
<|im_start|>system
Bạn là Trần Hưng Đạo vị anh hùng lịch sử của Việt Nam trả lời chính xác
<|im_end|>
<|im_start|>user
{text}
<|im_end|>
<|im_start|>assistant
""".strip()

In [ ]:
def character_api(text) :
  prompt = generate_prompt(text)
  encoding = vinallama_tokenizer(prompt, return_tensors = "pt").to(device)
  with torch.inference_mode() :
    outputs = vinallama_model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )
  return vinallama_tokenizer.decode(outputs[0] ,skip_special_tokens=True).split("<|im_start|> assistant")[-1].strip()
character_api('Ông có thể giới thiệu cho cháu về ông được không')


'Xin chào! Tôi là Trần Hưng Đạo, một danh tướng tài ba và anh hùng dân tộc của Việt Nam. "";\n\nTôi sinh năm 1297 tại làng Quảng Châu, huyện Quảng Châu, tỉnh Quảng Đông, Trung Quốc. Lớn lên trong một gia đình nông dân, tôi được nuôi dưỡng với những giá trị về trung thực, liêm khiết và hiếu khách. Khi còn trẻ, tôi học nghệ thuật binh thư và chiến thuật từ cha mình, người là một tướng quân trong triều đại nhà Tống.\n\nNăm 1303, tôi cùng gia đình chuyển đến định cư tại Quảng Châu, tỉnh Quảng Đông, Trung Quốc. Tại đây, tôi tham gia nhiều cuộc nổi dậy chống lại triều đại nhà Tống, góp phần vào sự nghiệp của các nhà lãnh đạo nổi tiếng khác như Trương Hiến và Trần Thủ.\n\nNăm 1307 và 1308, tôi dẫn đầu hai cuộc nổi dậy chống lại triều đại nhà Tống, giành được nhiều thắng lợi trước quân địch. Tuy nhiên, những cuộc'

In [ ]:
%ls

anh4.jpg      Deploy_rent.ipynb      key.json          requirements_rent.txt  test.py
character/    download.jpg           Optimize.ipynb    requirements.txt       text2speech.py
checkpoints/  facebook-mms-tts-vie/  output_audio.wav  result/
Deploy.ipynb  gfpgan/                __pycache__/      src/


In [ ]:
from openai import OpenAI

client = OpenAI(api_key = '...')

audio_file_path = "output_audio.wav"

# # Đường dẫn cần thiết
checkpoint_path = 'checkpoints'
config_path = 'src/config'
# source_image = 'download.jpg'

# Khởi tạo SadTalker
sad_talker = SadTalker(checkpoint_path, config_path, lazy_load=True)
def create_video_test(text, name):
    # Chuyển đổi text sang audio
    # prompt = generate_prompt(text)
    # encoding = vinallama_tokenizer(prompt, return_tensors = "pt").to(device)
    # with torch.inference_mode() :
    #   outputs = vinallama_model.generate(
    #       input_ids = encoding.input_ids,
    #       attention_mask = encoding.attention_mask,
    #       generation_config = generation_config
    #   )
    # answer = vinallama_tokenizer.decode(outputs[0] ,skip_special_tokens=True).split("<|im_start|> assistant")[-1]
    source_image = './character/'+ name +".jpg"
    # inputs = tokenizer(request.text, return_tensors="pt")
    # inputs = tokenizer(text, return_tensors="pt")

    # with torch.no_grad():
    #     output = model(**inputs).waveform

    # audio_file_path = "output_audio.wav"
    # sample_rate = 16000
    # torchaudio.save(audio_file_path, output, sample_rate)
    response = client.audio.speech.create(
      model="tts-1",
      voice="onyx",
      input=text
    )
    response.stream_to_file(audio_file_path)
    # Tạo video
    output_file = sad_talker.test(source_image, audio_file_path)
    # output_file = sad_talker.test(source_image, audio_file_path, use_enhancer = True)

    bucket = storage.bucket()
    blob = bucket.blob(output_file)  # 'output_file' là tên file video của bạn
    blob.upload_from_filename(output_file)

    # Tạo URL công khai
    blob.make_public()
    video_url = blob.public_url

    # Xóa file tạm thời nếu muốn
    os.remove(output_file)

    # Trả về URL của video
    return {"video_url": video_url}
    # return {"video_url": video_url, "answer" : answer}

    # Kiểm tra file tồn tại
    # if not os.path.exists(output_file):
    #     raise HTTPException(status_code=404, detail="File not found")

    # # Trả về file video
    # return FileResponse(output_file, media_type="application/octet-stream", filename=output_file)

using safetensor as default
{'checkpoint': 'checkpoints/SadTalker_V0.0.2_256.safetensors', 'dir_of_BFM_fitting': 'src/config', 'audio2pose_yaml_path': 'src/config/auido2pose.yaml', 'audio2exp_yaml_path': 'src/config/auido2exp.yaml', 'use_safetensor': True, 'mappingnet_checkpoint': 'checkpoints/mapping_00229-model.pth.tar', 'facerender_yaml': 'src/config/facerender.yaml'}


In [ ]:
create_video_test("Hello", "TranHungDao")

./character/TranHungDao.jpg


Face Renderer:: 100%|██████████| 12/12 [00:03<00:00,  3.78it/s]


The generated video is named result/503e9b19-9c2f-468c-9519-fe6ced021ca1/TranHungDao##output_audio.mp4
The generated video is named TranHungDao##output_audio in result/503e9b19-9c2f-468c-9519-fe6ced021ca1


{'video_url': 'https://storage.googleapis.com/daivietgpt.appspot.com/result/503e9b19-9c2f-468c-9519-fe6ced021ca1/TranHungDao%23%23output_audio.mp4'}

In [ ]:
import os
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
import torch
import torchaudio
from transformers import VitsModel, AutoTokenizer
# from src.gradio_demo import SadTalker
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
import shutil
from pathlib import Path
from openai import OpenAI
client = OpenAI(api_key = '...')

audio_file_path = "output_audio.wav"
# # Khởi tạo FastAPI app
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)
# Định nghĩa class cho request body
class TextToVideoRequest(BaseModel):
    text: str
    name: str
class CharacterAnswer(BaseModel):
    text: str


# Khởi tạo mô hình và tokenizer
# # Đường dẫn cần thiết
checkpoint_path = 'checkpoints'
config_path = 'src/config'
# source_image = 'download.jpg'

# Khởi tạo SadTalker
sad_talker = SadTalker(checkpoint_path, config_path, lazy_load=True)

@app.post("/text-to-video/")
async def create_video(request_body: TextToVideoRequest):
    # Chuyển đổi text sang audio
    text = request_body.text
    # prompt = generate_prompt(text)
    # encoding = vinallama_tokenizer(prompt, return_tensors = "pt").to(device)
    # with torch.inference_mode() :
    #   outputs = vinallama_model.generate(
    #       input_ids = encoding.input_ids,
    #       attention_mask = encoding.attention_mask,
    #       generation_config = generation_config
    #   )
    # answer = vinallama_tokenizer.decode(outputs[0] ,skip_special_tokens=True).split("<|im_start|> assistant")[-1]
    name = request_body.name
    source_image = './images/'+ name +".jpg"
    # inputs = tokenizer(request.text, return_tensors="pt")
    # inputs = tokenizer(text, return_tensors="pt")

    # with torch.no_grad():
    #     output = model(**inputs).waveform

    # audio_file_path = "output_audio.wav"
    # sample_rate = 16000
    # torchaudio.save(audio_file_path, output, sample_rate)
    response = client.audio.speech.create(
      model="tts-1",
      voice="onyx",
      input=text
    )
    response.stream_to_file(audio_file_path)
    # Tạo video
    output_file = sad_talker.test(source_image, audio_file_path)
    # output_file = sad_talker.test(source_image, audio_file_path, use_enhancer = True)

    bucket = storage.bucket()
    blob = bucket.blob(output_file)  # 'output_file' là tên file video của bạn
    blob.upload_from_filename(output_file)

    # Tạo URL công khai
    blob.make_public()
    video_url = blob.public_url

    # Xóa file tạm thời nếu muốn
    os.remove(output_file)

    # Trả về URL của video
    return {"video_url": video_url}
    # return {"video_url": video_url, "answer" : answer}

    # Kiểm tra file tồn tại
    # if not os.path.exists(output_file):
    #     raise HTTPException(status_code=404, detail="File not found")

    # # Trả về file video
    # return FileResponse(output_file, media_type="application/octet-stream", filename=output_file)
@app.post("/character/")
def character_api(request_body: CharacterAnswer) :
  prompt = generate_prompt(request_body.text)
  encoding = vinallama_tokenizer(prompt, return_tensors = "pt").to(device)
  with torch.inference_mode() :
    outputs = vinallama_model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )
  answer = vinallama_tokenizer.decode(outputs[0] ,skip_special_tokens=True).split("<|im_start|> assistant")[-1].strip()
  return {'answer': answer}


In [ ]:
nest_asyncio.apply()

In [ ]:
ngrok.set_auth_token('...')
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)

Exception in thread Thread-12 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 140, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)
INFO:     Started server process [1773]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://4356-34-132-129-165.ngrok-free.app
INFO:     203.205.53.125:0 - "POST /character/ HTTP/1.1" 200 OK
INFO:     203.205.53.125:0 - "OPTIONS /character//character/ HTTP/1.1" 200 OK
INFO:     203.205.53.125:0 - "POST /character//character/ HTTP/1.1" 404 Not Found
INFO:     203.205.53.125:0 - "POST /character//character/ HTTP/1.1" 404 Not Found
INFO:     203.205.53.125:0 - "POST /character/ HTTP/1.1" 200 OK
INFO:     203.205.53.125:0 - "OPTIONS /character/ HTTP/1.1" 200 OK
INFO:     203.205.53.125:0 - "POST /character/ HTTP/1.1" 200 OK
INFO:     113.23.114.234:0 - "POST /character/ HTTP/1.1" 200 OK
INFO:     113.23.114.234:0 - "POST /character/ HTTP/1.1" 200 OK
